In [ ]:
!unzip bluechip-summit-credit-worthiness-prediction.zip

Archive:  bluechip-summit-credit-worthiness-prediction.zip
  inflating: Sample Submission.csv   
  inflating: Test.csv                
  inflating: Train.csv               
  inflating: Variable_def.xlsx       


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("/content/Train.csv")
df.head()

,ID,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Total_Income
0,74768,LP002231,1,1,0,1,0,8328,0.000000,17,363,1,2,1,6000
1,79428,LP001448,1,1,0,0,0,150,3857.458782,188,370,1,1,0,6000
2,70497,LP002231,0,0,0,0,0,4989,314.472511,17,348,1,0,0,6000
3,87480,LP001385,1,1,0,0,0,150,0.000000,232,359,1,1,1,3750
4,33964,LP002231,1,1,1,0,0,8059,0.000000,17,372,1,0,1,3750


In [ ]:
df.shape

(5898, 15)

In [ ]:
df["Loan_Status"].value_counts()

,count
Loan_Status,
1,4913
0,985


In [ ]:
df.columns

Index(['ID', 'Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status',
       'Total_Income'],
      dtype='object')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5898 entries, 0 to 5897
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 5898 non-null   int64  
 1   Loan_ID            5898 non-null   object 
 2   Gender             5898 non-null   int64  
 3   Married            5898 non-null   int64  
 4   Dependents         5898 non-null   object 
 5   Education          5898 non-null   int64  
 6   Self_Employed      5898 non-null   int64  
 7   ApplicantIncome    5898 non-null   int64  
 8   CoapplicantIncome  5898 non-null   float64
 9   LoanAmount         5898 non-null   int64  
 10  Loan_Amount_Term   5898 non-null   int64  
 11  Credit_History     5898 non-null   int64  
 12  Property_Area      5898 non-null   int64  
 13  Loan_Status        5898 non-null   int64  
 14  Total_Income       5898 non-null   int64  
dtypes: float64(1), int64(12), object(2)
memory usage: 691.3+ KB


In [ ]:
df.isna().sum()

,0
ID,0
Loan_ID,0
Gender,0
Married,0
Dependents,0
Education,0
Self_Employed,0
ApplicantIncome,0
CoapplicantIncome,0
LoanAmount,0


In [ ]:
pd.crosstab(df.Dependents,df.Loan_Status)

Loan_Status,0,1
Dependents,,
0,587,3072
1,203,954
2,95,441
3+,100,446


In [ ]:
#from sklearn.preprocessing import LabelEncoder
#le = LabelEncoder()
#df['Loan_ID'] = le.fit_transform(df['Loan_ID'])
#df["Dependents"] = le.fit_transform(df['Dependents'])

In [ ]:
X = df.drop("Loan_Status", axis = 1)
y = df["Loan_Status"]

In [ ]:
np.random.seed(42)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_test = pd.DataFrame(OH_encoder.transform(X_test[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_test.index = X_test.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_test = X_test.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_test = pd.concat([num_X_test, OH_cols_test], axis=1)

# Ensure all columns have string type
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_test.columns = OH_X_test.columns.astype(str)

X_train = OH_X_train
X_test = OH_X_test

In [ ]:
len(X_train)

4718

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from xgboost import XGBClassifier
models = {"Logistic Regression": LogisticRegression(),
          "XG Boost" : XGBClassifier(),
          "Random Forest": RandomForestClassifier()}
def fit_and_score(models, X_train, X_test, y_train, y_test):
    """
    Fits and evaluates given machine learning models.
    models : a dict of different Scikit-Learn machine learning models
    X_train : training data (no labels)
    X_test : testing data (no labels)
    y_train : training labels
    y_test : test labels
    """
    # Set Random seed
    np.random.seed(42)
    # Make a dictionary to keep model scores
    model_scores = {}
    # Loop through models
    for name, model in models.items():
        # Fit the model to the data
        model.fit(X_train, y_train)
        # Evaluate the model and append its score to model_scores
        model_scores[name] = model.score(X_test, y_test)
    return model_scores

In [ ]:
model_scores = fit_and_score(models=models,
                             X_train=X_train,
                             X_test=X_test,
                             y_train=y_train,
                             y_test=y_test)

model_scores

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'Logistic Regression': 0.8415254237288136,
 'XG Boost': 0.8177966101694916,
 'Random Forest': 0.8398305084745763}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
model_ = RandomForestClassifier()
param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': np.arange(1, 10)
}
random_search = RandomizedSearchCV(estimator=model_, param_distributions=param_dist, n_iter=20, cv=3)

In [ ]:
random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=20,
                   param_distributions={'max_depth': [10, 20, 30],
                                        'min_samples_leaf': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300]})

In [ ]:
print(random_search.best_params_)

{'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 6, 'max_depth': 30}


In [ ]:
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(y_test,random_search.predict(X_test)))


0.15847457627118644


In [ ]:
ideal_model = RandomForestClassifier(n_estimators=297,
                                    min_samples_split=6,
                                    min_samples_leaf = 6,
                                    max_depth = 45,

              )
ideal_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=45, min_samples_leaf=6, min_samples_split=6,
                       n_estimators=297)

In [ ]:
print(mean_absolute_error(y_test,ideal_model.predict(X_test)))


0.15847457627118644


In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, ideal_model.predict(X_test))
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.84


In [ ]:
from sklearn.model_selection import GridSearchCV
modeller = LogisticRegression()
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l2', 'none'],  # Options depend on the solver
    'solver': ['lbfgs', 'saga'],  # Ensure compatibility with penalty
    'max_iter': [100, 200, 500]
}
grid_search = GridSearchCV(estimator=modeller, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.01, 0.1, 1, 10, 100],
                         'max_iter': [100, 200, 500], 'penalty': ['l2', 'none'],
                         'solver': ['lbfgs', 'saga']})

In [ ]:
print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'C': 0.01, 'max_iter': 100, 'penalty': 'l2', 'solver': 'saga'}


In [ ]:
print(mean_absolute_error(y_test,grid_search.predict(X_test)))

0.15847457627118644


In [ ]:
real_model = LogisticRegression(C=0.01,
                                    max_iter=100,
                                    penalty = 'l2',
                                    solver= 'saga',

              )
real_model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(C=0.01, solver='saga')

In [ ]:
print(mean_absolute_error(y_test,real_model.predict(X_test)))

0.15847457627118644


In [ ]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.4 MB/s eta 0:00:00


In [ ]:
import optuna
from sklearn.model_selection import cross_val_score
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 500)
    max_depth = trial.suggest_int('max_depth', 10, 50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    model_s = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split
    )
    return cross_val_score(model_s, X_train, y_train, cv=3).mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)
print(study.best_params)

[I 2024-11-27 08:08:24,526] A new study created in memory with name: no-name-ca451040-d55f-4fa2-957b-9aeeaf0af078
[I 2024-11-27 08:08:38,310] Trial 0 finished with value: 0.8308605189243634 and parameters: {'n_estimators': 483, 'max_depth': 13, 'min_samples_split': 8}. Best is trial 0 with value: 0.8308605189243634.
[I 2024-11-27 08:08:46,478] Trial 1 finished with value: 0.830648609621545 and parameters: {'n_estimators': 420, 'max_depth': 43, 'min_samples_split': 8}. Best is trial 0 with value: 0.8308605189243634.
[I 2024-11-27 08:08:54,762] Trial 2 finished with value: 0.830648609621545 and parameters: {'n_estimators': 360, 'max_depth': 32, 'min_samples_split': 5}. Best is trial 0 with value: 0.8308605189243634.
[I 2024-11-27 08:09:01,639] Trial 3 finished with value: 0.8308605189243634 and parameters: {'n_estimators': 379, 'max_depth': 17, 'min_samples_split': 10}. Best is trial 0 with value: 0.8308605189243634.
[I 2024-11-27 08:09:07,856] Trial 4 finished with value: 0.830648609621

{'n_estimators': 297, 'max_depth': 45, 'min_samples_split': 6}


In [ ]:
models = RandomForestClassifier(
    n_estimators=100,        # Number of trees
    max_depth=10,            # Limit tree depth
    min_samples_split=5,     # Minimum samples to split a node
    min_samples_leaf=2,      # Minimum samples in each leaf
    max_features='sqrt',     # Use sqrt of features for splitting
    bootstrap=True,          # Use bootstrapped samples
    random_state=42          # For reproducibility
)
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}
grid_searching = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    scoring='accuracy',  # Adjust scoring based on your metric
    cv=5,
    n_jobs=-1
)
grid_searching.fit(X_train, y_train)
print("Best Parameters:", grid_searching.best_params_)
print("Best Accuracy:", grid_searching.best_score_)

Best Parameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best Accuracy: 0.8310730718766288


In [ ]:
td = pd.read_csv("/content/Test.csv")

In [ ]:
td

,ID,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Total_Income
0,70607,LP002560,1,1,0,1,0,15890,871.075952,188,371,1,1,6000
1,58412,LP001379,1,1,0,0,1,6582,896.718887,17,373,0,1,6000
2,88755,LP002560,0,0,0,0,0,7869,572.900354,17,373,1,1,6000
3,97271,LP002560,1,1,0,0,0,150,0.000000,247,349,1,2,6000
4,70478,LP002231,1,1,0,0,0,8362,0.000000,17,12,1,2,3750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2523,15578,LP002175,1,1,0,0,0,7276,0.000000,168,367,1,1,3750
2524,87689,LP002582,1,1,0,0,1,8153,1585.948119,17,406,1,1,3750
2525,42584,LP002231,1,1,0,0,0,16382,600.813513,17,348,1,2,3750
2526,44709,LP002224,1,1,0,0,0,7791,0.000000,78,350,0,1,3750


In [ ]:
td.isna().sum()

,0
ID,0
Loan_ID,0
Gender,0
Married,0
Dependents,0
Education,0
Self_Employed,0
ApplicantIncome,0
CoapplicantIncome,0
LoanAmount,0


In [ ]:
s = (td.dtypes == 'object')
categorical_columns = list(s[s].index)
X_new_encoded = pd.get_dummies(td, columns=categorical_columns)
X_new_encoded = X_new_encoded.reindex(columns=X_train.columns, fill_value=0)


In [ ]:
#from sklearn.preprocessing import LabelEncoder
#le = LabelEncoder()
#td['Loan_ID'] = le.fit_transform(td['Loan_ID'])
#td["Dependents"] = le.fit_transform(td['Dependents'])

In [ ]:
predictions = real_model.predict(X_new_encoded)

In [ ]:
#td['Loan_ID'] = le.inverse_transform(td["Loan_ID"])
#td['"Dependents"'] = le.inverse_transform(td["Dependents"])

ValueError: y contains previously unseen labels: [  4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20  21
  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39
  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57
  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75
  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93
  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111
 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129
 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147
 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165
 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183
 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201
 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219
 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237
 2

In [ ]:
print(predictions)

[1 1 1 ... 1 1 1]


In [ ]:
td["Loan_Status"] = predictions
print(td[['ID', "Loan_Status"]])

         ID  Loan_Status
0     70607            1
1     58412            1
2     88755            1
3     97271            1
4     70478            1
...     ...          ...
2523  15578            1
2524  87689            1
2525  42584            1
2526  44709            1
2527  75533            1

[2528 rows x 2 columns]


In [ ]:
data = td[['ID', "Loan_Status"]]
data.to_csv('submission_file.csv', index=False)